# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [1]:
# Your code here
!ls -ltr



total 516
-rw-r--r-- 1 poari 197609   1849 Sep 26 18:08 CONTRIBUTING.md
-rw-r--r-- 1 poari 197609   1371 Sep 26 18:08 LICENSE.md
-rw-r--r-- 1 poari 197609   2355 Sep 26 18:08 README.md
-rw-r--r-- 1 poari 197609 483481 Sep 26 18:08 SMSSpamCollection
-rw-r--r-- 1 poari 197609  27188 Oct  1 07:01 index.ipynb


In [2]:

import pandas as pd
df= pd.read_csv('SMSSpamCollection',names= ['label','text'],sep='\t')
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [3]:
# Your code here
import numpy as np
print(df['label'].value_counts())

# extract the ham rows in a separate df
df_ham=df[df['label']=='ham']

# extract the spam rows in a separate df
df_spam=df[df['label']=='spam']

# reset index of df_ham
df_ham.reset_index(inplace = True)

# generate 747 random numbers in the interval 0-4824
indices = np.random.randint(0, 4825, 747)

# select from df_ham
df_ham_select=df_ham.iloc[list(indices)]

# put together the selected ham records and the spam records
df_balanced = pd.concat([df_ham_select,df_spam],axis=0,sort=True)
df_balanced

ham     4825
spam     747
Name: label, dtype: int64


,index,label,text
2913,3359.0,ham,I don't run away frm u... I walk slowly &amp; ...
557,652.0,ham,I'm gonna say no. Sorry. I would but as normal...
1491,1737.0,ham,I will come tomorrow di
151,182.0,ham,Lol no. U can trust me.
3984,4607.0,ham,Oh... Haha... Den we shld had went today too.....
...,...,...,...
5537,NaN,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,NaN,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,NaN,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,NaN,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [4]:
df_balanced.reset_index(inplace=True)


In [5]:
df_balanced


,level_0,index,label,text
0,2913,3359.0,ham,I don't run away frm u... I walk slowly &amp; ...
1,557,652.0,ham,I'm gonna say no. Sorry. I would but as normal...
2,1491,1737.0,ham,I will come tomorrow di
3,151,182.0,ham,Lol no. U can trust me.
4,3984,4607.0,ham,Oh... Haha... Den we shld had went today too.....
...,...,...,...,...
1489,5537,NaN,spam,Want explicit SEX in 30 secs? Ring 02073162414...
1490,5540,NaN,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
1491,5547,NaN,spam,Had your contract mobile 11 Mnths? Latest Moto...
1492,5566,NaN,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [6]:
print(df_balanced.columns)
df_balanced.drop(['level_0','index'],axis=1,inplace=True)
print(df_balanced.columns)

Index(['level_0', 'index', 'label', 'text'], dtype='object')
Index(['label', 'text'], dtype='object')


## Train-test split

Now implement a train-test split on the dataset: 

In [7]:
# Your code here
from sklearn.model_selection import train_test_split
# remove class label from df_balanced
X = df_balanced.drop(columns=['label'],axis = 1)
y = df_balanced['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)
train_df = pd.concat([X_train,y_train],axis=1)
test_df = pd.concat([X_test,y_test],axis=1)

In [8]:
print(train_df.shape,test_df.shape)

(1195, 2) (299, 2)


In [9]:
299/(299+1195)


0.2001338688085676

In [10]:
train_df.columns

Index(['text', 'label'], dtype='object')

In [11]:
list(train_df['label'].unique())

['spam', 'ham']

In [12]:
train_df.index

Int64Index([1341,  226,   33,  239,  707,  847,   54,  552,  769,  728,
            ...
            1266,  406,  759,  935,  845,  522, 1143, 1278, 1102,  771],
           dtype='int64', length=1195)

In [13]:
train_df.head()

,text,label
1341,Ur TONEXS subscription has been renewed and yo...,spam
226,"Like &lt;#&gt; , same question",ham
33,S:-)if we have one good partnership going we w...,ham
239,Hi.:)technical support.providing assistance to...,ham
707,Who u talking about?,ham


In [14]:
train_df.iloc[1]

text     Like  &lt;#&gt; , same question
label                                ham
Name: 226, dtype: object

In [15]:
train_df.index

Int64Index([1341,  226,   33,  239,  707,  847,   54,  552,  769,  728,
            ...
            1266,  406,  759,  935,  845,  522, 1143, 1278, 1102,  771],
           dtype='int64', length=1195)

In [16]:
train_df.iloc[2]

text     S:-)if we have one good partnership going we w...
label                                                  ham
Name: 33, dtype: object

In [17]:
train_df['text'][1413].split()

['Hi',
 'this',
 'is',
 'Amy,',
 'we',
 'will',
 'be',
 'sending',
 'you',
 'a',
 'free',
 'phone',
 'number',
 'in',
 'a',
 'couple',
 'of',
 'days,',
 'which',
 'will',
 'give',
 'you',
 'an',
 'access',
 'to',
 'all',
 'the',
 'adult',
 'parties...']

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [18]:
# Your code here

# Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq} 
# will be based on training set only

# list of classes ham/spam
classes=list(train_df['label'].unique())
dico={}
for class_ in classes:
    # temporary dataframe containing only class_ 
    temp_df=train_df[train_df['label']==class_]
    # initialize an empty dictionary for bag of words
    bag={}
    # need to go through each row, take the text, split it and add up each word's count
    for row in temp_df.index:
        texto = temp_df['text'][row]
        for word in texto.split():
            # look for each word in the keys of dictionary bag. 
            # if found, return the value of word + 1
            # if not found, return  +1
            bag[word]=bag.get(word,0)+1
    dico[class_]=bag
dico

{'spam': {'Ur': 12,
  'TONEXS': 1,
  'subscription': 3,
  'has': 22,
  'been': 37,
  'renewed': 1,
  'and': 93,
  'you': 135,
  'have': 109,
  'charged': 11,
  '£4.50.': 3,
  'You': 63,
  'can': 20,
  'choose': 2,
  '10': 4,
  'more': 14,
  'polys': 4,
  'this': 32,
  'month.': 1,
  'www.clubzed.co.uk': 1,
  '*BILLING': 1,
  'MSG*': 1,
  'To': 55,
  'review': 4,
  'KEEP': 1,
  'the': 143,
  'fantastic': 3,
  'Nokia': 35,
  'N-Gage': 1,
  'game': 1,
  'deck': 1,
  'with': 84,
  'Club': 3,
  'Nokia,': 1,
  'go': 8,
  '2': 136,
  'www.cnupdates.com/newsletter.': 1,
  'unsubscribe': 12,
  'from': 97,
  'alerts': 1,
  'reply': 46,
  'word': 10,
  'OUT': 5,
  'Sunshine': 7,
  'Quiz': 7,
  'Wkly': 4,
  'Q!': 4,
  'Win': 11,
  'a': 291,
  'top': 11,
  'Sony': 5,
  'DVD': 6,
  'player': 9,
  'if': 16,
  'u': 37,
  'know': 19,
  'which': 8,
  'country': 4,
  'Algarve': 2,
  'is': 114,
  'in?': 2,
  'Txt': 54,
  'ansr': 4,
  'to': 477,
  '82277.': 5,
  '£1.50': 15,
  'SP:Tyrone': 4,
  'This': 32,

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [28]:
# Your code here
# create an empty set:
corpus_train = set()
for row in train_df.index:
    for word in train_df['text'][row].split():
        corpus_train.add(word)
corpus_train
V=len(corpus_train)
V

6131

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [ ]:
# Your code here

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [ ]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    pass

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [ ]:
# Your code here


## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!